# Code Workshop TDC DEMO



## setup

* Elastic Credentials - Create an [Elastic Cloud deployment](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud) to get all Elastic credentials (`ELASTIC_CLOUD_ID`, `ELASTIC_API_KEY`).



## Install packages

In [24]:
!pip install -q -U python-dotenv

In [25]:
!pip install -q -U install langchain langchain-community pypdf tiktoken langchain_openai langchain_elasticsearch

## Import packages and credentials

In [26]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


## Get Credentials

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import os
from dotenv import load_dotenv

# Substitua 'path/to/your/.env' pelo caminho correto até o seu arquivo .env no Google Drive
env_path="/content/drive/MyDrive/@eventos/2024/TDCFloripa/env_.txt"
load_dotenv(env_path)

True

In [29]:

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

In [30]:
# Elastic cloud credentials
es_cloud_id = os.getenv('cloud_id')
es_user = os.getenv('cloud_user')
es_pass = os.getenv('cloud_pass')

CLOUD_USER=es_user
CLOUD_PASS=es_pass
CLOUD_ID=es_cloud_id

ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY')


In [31]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("/content/drive/MyDrive/@eventos/2024/TDCFloripa/arquivos/concurso-bnb-edital-1-2022.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [32]:
type(docs[0])

langchain_core.documents.base.Document

In [33]:
# Split
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 1500,
#     chunk_overlap = 500
# )

# Testar com split de tokens
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(
    chunk_size = 500,
    chunk_overlap = 30
)

In [34]:
splits = text_splitter.split_documents(docs)

In [35]:
len(splits)

114

In [36]:
from langchain_openai import OpenAIEmbeddings


In [37]:
ELASTIC_INDEX_NAME="tdc-workshop-003"

In [38]:
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import ChatOpenAI


In [39]:
query_embedding=OpenAIEmbeddings()


In [40]:
#query_embedding = GoogleGenerativeAIEmbeddings(
#    model="models/embedding-001", task_type="retrieval_document"
#)
es = ElasticsearchStore.from_documents(
    splits,
    es_cloud_id=CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name=ELASTIC_INDEX_NAME,
    embedding=query_embedding,
)

In [41]:
def format_docs(docs):
    return " ".join(doc.page_content for doc in docs)
    ##return "\n\n".join(doc.page_content for doc in docs)

In [42]:
retriever = es.as_retriever(search_kwargs={"k": 3})     #outro ponto de ajuste


In [43]:

#     | ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.8)

template = """Answer the question in portuguese based only on the following context:\n

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | ChatOpenAI(model="gpt-4o") ## Calling Gpt-4o
    | StrOutputParser()
)



In [44]:
def print_chat(text):
    # Substitui os caracteres de nova linha e os prompts interrompidos
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [45]:
question1 = "Qual o concurso disponivel"
chain.invoke(question1)


'O concurso disponível é para os cargos de Especialista Técnico – Analista de Sistemas, com os perfis de Desenvolvimento de Sistemas e Infraestrutura e Segurança da Informação.'

## Add documents

Ler PDF

### Let's download the sample dataset and deserialize the document.

In [ ]:
# PDF

**Storage**

# Retrieval

In [49]:
question2 = "qual a faixa salarial"
chain.invoke(question2)

'A faixa salarial inicial é de R$ 6.269,76, composta por um vencimento base de R$ 4.702,32 acrescido de uma gratificação mensal de R$ 1.567,44 durante o contrato de experiência. Após o término do contrato de experiência, com resultado satisfatório, o ocupante do cargo terá direito a uma remuneração adicional pelo exercício de função em comissão, conforme a regulamentação interna.'

In [48]:
question3 = "Fale mais sobre o cargo Especialista Técnico – Analista de Sistemas – Perfil 1: Desenvolvimento de Sistemas"
chain.invoke(question3)


'O cargo de **Especialista Técnico – Analista de Sistemas – Perfil 1: Desenvolvimento de Sistemas** é uma posição voltada para o desenvolvimento de sistemas, conforme descrito no contexto fornecido. Aqui estão alguns detalhes adicionais sobre esse cargo:\n\n1. **Remuneração**:\n   - Durante o contrato de experiência, a remuneração é de R$ 4.702,32, acrescida de um terço de gratificação mensal no valor de R$ 1.567,44, totalizando uma remuneração inicial de R$ 6.269,76.\n   - Após o término do contrato de experiência, desde que o desempenho seja satisfatório, o ocupante do cargo terá direito a uma remuneração adicional pelo exercício da função em comissão, conforme regulamentação interna.\n\n2. **Jornada de Trabalho**:\n   - A jornada de trabalho é de 30 horas semanais.\n\n3. **Requisitos Básicos para Contratação**:\n   - Ser aprovado no concurso público.\n   - Ter nacionalidade brasileira ou portuguesa (no caso de nacionalidade portuguesa, deve estar amparado pelo Estatuto de Igualdade 

In [ ]:
question4 = "quais os pre-requisitos?"
chain.invoke(question4)

'Os pré-requisitos para o ingresso nos quadros do BNB, conforme o edital, são:\n\n1. **Não ter praticado crimes**:\n   - Comprovada prática, nos dois anos anteriores à data prevista para a posse, dos crimes definidos na Lei nº 9.613, de 3 de março de 1998, como crime de lavagem de dinheiro ou ocultação de bens, direitos e valores.\n   - Outros atos legalmente definidos como crimes contra o Sistema Financeiro Nacional ou a Ordem Econômica Nacional.\n\n2. **Autorização para esclarecimentos**:\n   - Concessão de autorização irrestrita e irrevogável para que o Banco possa solicitar a terceiros e para que esses possam prestar esclarecimentos sobre todos os fatos relacionados às informações prestadas.\n\n3. **Preenchimento de documentos internos**:\n   - Preencher outros documentos internos necessários na qualificação para a contratação, conforme modelos disponibilizados pelo BNB.\n\n4. **Resultado positivo no processo de qualificação**:\n   - Obtenção de resultado positivo no processo de qu

In [ ]:
# verificar ajustes de tokens de saida

In [50]:
!pip install openai

# Question Answerings

In [ ]:
# Question Answerings

## Retrieval Chain types

In [51]:

# Run chain
from langchain.chains import RetrievalQA
question = "quais cargos abertos no concurso?"


In [52]:
llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0)
llm.predict("Bom dia!")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Bom dia! Como posso ajudar você hoje?'

In [53]:
# map_reduce
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type="map_reduce"
)

In [54]:
result = qa_chain_mr({"query": question})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [55]:
result["result"]

'Desculpe, mas não consigo responder a essa pergunta com base no trecho fornecido. Posso ajudar com mais alguma coisa?'

In [56]:
# refine
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'Com base no novo contexto fornecido, os cargos abertos no concurso são:\n\n- Cargo 1: Especialista Técnico - Analista de Sistemas - Perfil 1: Desenvolvimento de Sistemas\n- Cargo 2: Especialista Técnico - Analista de Sistemas - Perfil 2: Infraestrutura e Segurança da Informação\n\nPara o Cargo 1, há 90 vagas para ampla concorrência, 6 vagas reservadas para candidatos com deficiência e 24 vagas reservadas para candidatos negros.\n\nPara o Cargo 2, há 60 vagas para ampla concorrência, 4 vagas reservadas para candidatos com deficiência e 16 vagas reservadas para candidatos negros. No entanto, apenas os 10 candidatos mais bem classificados nas provas objetivas serão convocados para este cargo.\n\nOs candidatos contratados serão lotados na sede do BNB, na cidade de Fortaleza/CE.'

In [ ]:
# refine
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type="map_rerank"
)
result = qa_chain_mr({"query": question})
result["result"]

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:367: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


'Cargo 2: Especialista Técnico - Analista de Sistemas - Perfil 2: Infraestrutura e Segurança da Informação'

# Memory


In [57]:
# aplicando memoria
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [58]:
from langchain.chains import ConversationalRetrievalChain
retriever=retriever
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [59]:
result = qa({"question": question})

In [60]:
result['answer']

'Os cargos abertos no concurso são:\n\n1. Cargo 1: Especialista Técnico – Analista de Sistemas – Perfil 1: Desenvolvimento de Sistemas\n2. Cargo 2: Especialista Técnico – Analista de Sistemas – Perfil 2: Infraestrutura e Segurança da Informação\n\nEspero que essa informação seja útil!'

In [61]:
question2= "Fale mais sobre o Cargo 1: Especialista Técnico – Analista de Sistemas – Perfil 1: Desenvolvimento de Sistemas"

In [62]:
question = question2
result = qa({"question": question})

In [63]:
result['answer']

'No documento fornecido, o Cargo 1: Especialista Técnico – Analista de Sistemas – Perfil 1: Desenvolvimento de Sistemas possui 90 vagas para ampla concorrência, 6 vagas reservadas para candidatos com deficiência e 24 vagas reservadas para candidatos negros. Os candidatos contratados para esse cargo serão lotados na sede do BNB, na cidade de Fortaleza/CE. Além disso, durante o contrato de experiência, a remuneração para esse cargo é de R$ 6.269,76, com uma jornada de trabalho de 30 horas semanais.'

In [64]:
question = "Qual o salario para esse cargo"
result = qa({"question": question})

In [65]:
result['answer']

'Durante o contrato de experiência, o salário para o Cargo 1 é de R$ 6.269,76, sendo R$ 4.702,32 como vencimento do cargo e R$ 1.567,44 de gratificação mensal. Após o término do contrato de experiência, com resultado satisfatório, o ocupante do cargo terá direito a uma remuneração adicional do exercício de função em comissão, conforme a regulamentação interna.'